---

Here use your CSV from WIX: Example! "Pole Studios (2).csv", replace the Name of the CSV inside the "", Run All Cells!

In [1]:
import pandas as pd

# Load initial URLs
hop_dataframe = pd.read_csv("Pole Studios (2).csv")
hop_dataframe.head(3)

,Pole Studio,PLZ,Stadt,Land,Adresse,Telefonnummer,E-Mail-Adresse,Webseite,Facebook,Instagram,...,ID,Owner,Created Date,Updated Date,Bildergalerie,Richtigkeit der Angaben,Pole Studios (Pole Studio),Online Studios (Pole Studio),Inaktiv,Löschvermerk
0,Seemannsbraut Poledance Flensburg,24941,Flensburg,Deutschland,"{""subdivisions"":[{""code"":""SH"",""name"":""Schleswi...",15163300769,seemannsbraut-poledance@web.de,https://www.seemannsbraut-poledance.de/,https://www.facebook.com/SeemannsbrautPoledance,https://www.instagram.com/seemannsbraut_poleda...,...,01df35bd-209a-4520-a7bc-d109b9aaf2f6,284241e1-0e53-4183-b67f-34edcfb7c8b1,2022-11-18T11:57:31Z,2024-02-04T18:00:48Z,NaN,NaN,/polestudio/seemannsbraut-poledance-flensburg,/onlinestudio/seemannsbraut-poledance-flensburg,NaN,NaN
1,TSG Blau Gold Ballett- und Pole-dance- Studio,35390,Gießen,Deutschland,"{""subdivisions"":[{""code"":""HE"",""name"":""Hessen"",...",0491 621606468,tanz-giessen@gmx.de,https://www.tsg-giessen.de/angebote/tanz/poled...,https://www.facebook.com/poledancegiessen.de,https://www.instagram.com/tsg_blaugoldgiessen....,...,0233997c-a277-4179-990a-347e1cea347e,284241e1-0e53-4183-b67f-34edcfb7c8b1,2022-11-18T11:57:31Z,2024-02-04T18:00:49Z,NaN,NaN,/polestudio/tsg-blau-gold-ballett--und-pole-da...,/onlinestudio/tsg-blau-gold-ballett--und-pole-...,NaN,NaN
2,Pole Stars Osnabrück,49080,Osnabrück,Deutschland,"{""subdivisions"":[{""code"":""NDS"",""name"":""Nieders...",01573 1810262,info@polestars-osnabrueck.de,http://www.polestars-osnabrueck.de/,https://www.facebook.com/PoleStarsOsnabrueck.de/,https://www.instagram.com/pole_stars_osnabruec...,...,04673b9d-d8b2-4caf-8632-d349eab2cd4e,284241e1-0e53-4183-b67f-34edcfb7c8b1,2022-11-18T11:57:31Z,2024-02-04T18:00:48Z,NaN,NaN,/polestudio/pole-stars-osnabr%C3%BCck,/onlinestudio/pole-stars-osnabr%C3%BCck,NaN,NaN


---

Url Reconstruction

In [2]:
hop_polestudios = hop_dataframe["Pole Studios (Pole Studio)"][:20].to_list()

initial_urls = hop_polestudios

base_url = "https://www.eversports.de/s"

#initial_urls = list(set(hop_dataframe["Pole Studios (Pole Studio)"]))
len(initial_urls)

20

In [3]:
url = []
for url_part in initial_urls:
    url.append(base_url+url_part[11:])
url[:10]

['https://www.eversports.de/s/seemannsbraut-poledance-flensburg',
 'https://www.eversports.de/s/tsg-blau-gold-ballett--und-pole-dance--studio',
 'https://www.eversports.de/s/pole-stars-osnabr%C3%BCck',
 'https://www.eversports.de/s/aerial-art-studio',
 'https://www.eversports.de/s/k%C3%B6rpernah',
 'https://www.eversports.de/s/dance-moves-wolfenb%C3%BCttel',
 'https://www.eversports.de/s/mypole',
 'https://www.eversports.de/s/tanzstudio-michel',
 'https://www.eversports.de/s/pole-sensation',
 'https://www.eversports.de/s/poledance-rostock']

In [4]:
initial_urls = url

---

Scraper Code

In [5]:
import pandas as pd
from a_PyCaller import process_urls
from tqdm import tqdm
from datetime import datetime

def process_and_print_results(url, all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data):
    data = process_urls([url])

    if data:
        for key, df in data.items():
            if df is not None and not df.empty:
                tqdm.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - INFO - Scraping Data from: {url}")
                tqdm.write(f"{key.replace('_', ' ').title()}: {len(df)} entries")

                # Update the appropriate DataFrame
                if key == 'pole_studio_data':
                    all_pole_studio_data = pd.concat([all_pole_studio_data, df], ignore_index=True)
                elif key == 'workshops_data':
                    all_workshops_data = pd.concat([all_workshops_data, df], ignore_index=True)
                elif key == 'workshop_details':
                    all_workshop_details_data = pd.concat([all_workshop_details_data, df], ignore_index=True)

    # Add URLs to DataFrame
    all_urls_data = pd.concat([all_urls_data, pd.DataFrame({'URL': [url]})], ignore_index=True)

    return all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data

def main():
    # # Load initial URLs
    # all_found_urls_s = pd.read_csv("your_output_file.csv")
    # initial_urls = list(set(all_found_urls_s["0"])) # Delete "[:3]" to scrape all Urls

    # Initialize DataFrames
    all_pole_studio_data = pd.DataFrame()
    all_workshops_data = pd.DataFrame()
    all_workshop_details_data = pd.DataFrame()
    all_urls_data = pd.DataFrame(columns=['URL'])

    # Process each URL with tqdm
    with tqdm(initial_urls, desc="Processing URLs", dynamic_ncols=True) as pbar:
        for url in pbar:
            all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data = process_and_print_results(
                url, all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data
            )

    # Export DataFrames to CSV files
    # all_pole_studio_data.to_csv("Pole_Studio_Übersicht_S.csv", index=False)
    # all_workshops_data.to_csv("Workshop_Liste_SW.csv", index=False)
    # all_workshop_details_data.to_csv("Workshop_Übersicht_E.csv", index=False)
    # all_urls_data.to_csv("All_URLs.csv", index=False)

    # Return the final DataFrames
    return all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data

# Run the main function and get the final DataFrames
result_pole_studio, result_workshops, result_workshop_details, result_urls = main()


Processing URLs:   0%|          | 0/20 [00:00<?, ?it/s]

Starting URL reconstruction...


Processing URLs:   5%|▌         | 1/20 [00:02<00:48,  2.53s/it]

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/seemannsbraut-poledance-flensburg
Converting to DataFrame...
Validating URLs...

Processing completed.
-------------------------------- 

Starting URL reconstruction...


Processing URLs:  10%|█         | 2/20 [00:03<00:31,  1.76s/it]

...


Processing completed.
-------------------------------- 

2024-02-20 16:17:37 - INFO - Scraping Data from: https://www.eversports.de/s/luftbonbon
Pole Studio Data: 1 entries
2024-02-20 16:17:37 - INFO - Scraping Data from: https://www.eversports.de/s/luftbonbon
Workshops Data: 1 entries
2024-02-20 16:17:37 - INFO - Scraping Data from: https://www.eversports.de/s/luftbonbon
Workshop Details: 1 entries
Starting URL reconstruction...


Processing URLs:  95%|█████████▌| 19/20 [00:57<00:03,  3.14s/it]

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/exquisuite
Converting to DataFrame...
Validating URLs...

Processing completed.
-------------------------------- 

Starting URL reconstruction...


Processing URLs: 100%|██████████| 20/20 [00:58<00:00,  2.92s/it]

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/pole-emotion-bremerhaven
Converting to DataFrame...
Validating URLs...

Processing completed.
-------------------------------- 



In [6]:
result_pole_studio

,PoleStudio_Name,Adresse,PLZ,Stadt,Straße,Buttons,Pole Studio Beschreibung,E-Mail,Homepage,Telefon,URL_S,Art,Angebot,Created Date,Updated Date
0,Tanzstudio Michel,"[Schmalweg 50 , 55252 Wiesbaden]",55252,Wiesbaden,Schmalweg 50,"[Übersicht, Klassen, Preise, Team]",Das etablierte Tanzstudio in Wiesbaden bietet ...,t.michel.9@gmx.de,https://www.tanzstudio-michel.de/,+49 15115125871,https://www.eversports.de/s/tanzstudio-michel,"[1 Rezensionen, Fitness, Tanzen, Poledance, St...",Zum Stundenplan,2024-02-20 16:17:02,2024-02-20 16:17:02
1,Polestation Academy,"[Marktstraße 20b , 6971 Hard]",6971,Hard,Marktstraße 20b,"[Übersicht, Klassen, Videos, Preise, Team]",Polestation Academy ist ein Ort für Pole und A...,blum.n@gmx.at,http://www.polestation.com,+43 6642006568,https://www.eversports.de/s/polestation-academy,"[Poledance, Poledance, Poledance, Poledance, P...",Zum Stundenplan,2024-02-20 16:17:22,2024-02-20 16:17:22
2,Poledance 4 you,"[Naugarder Straße 14, 10409 Berlin]",10409,Berlin,Naugarder Straße 14,[Übersicht],Das Motto des Berliner Pole Dance Studios laut...,info@poledance4you.de,https://www.poledance4you.de/,None,https://www.eversports.de/s/poledance-4-you,"[(Noch keine Bewertungen), Poledance]",None,2024-02-20 16:17:30,2024-02-20 16:17:30
3,Luftbonbon,"[Königstraße 45 , 32547 Bad Oeynhausen]",32547,Bad,Königstraße 45,"[Übersicht, Klassen, Workshops, Preise, Team]",Entdecke mit uns die vielseitige Welt der Luft...,info@luftbonbon.com,https://www.luftbonbon.com/,01622572081,https://www.eversports.de/s/luftbonbon,"[Poledance, Aerial Hoop, Poledance, Stärkung/F...",Zum Stundenplan,2024-02-20 16:17:34,2024-02-20 16:17:34


In [7]:
result_workshops

,Workshop Name,Workshop Date,Workshop Price,Workshop Units,Workshop Studio,Workshop Address,URL_E
0,Spagat,Feb05bis Feb 26,Ausgebucht,4 Einheiten,Luftbonbon,"Königstraße 45, 32547 Bad Oeynhausen",https://www.eversports.de/e/workshop/60kU5cU


In [8]:
result_workshop_details

,Workshop Name,Description,Studio Name,Location,Art,Date,URL_E,Time
0,Spagat,Straddle & Middlesplit stehen ganz oben auf De...,Luftbonbon,20:20 - 21:20,Stärkung/Flexibilität,"Begann am Montag, 5. Februar 2024",https://www.eversports.de/e/workshop/60kU5cU,"Königstraße 45, 32547 Bad Oeynhausen"


In [9]:
result_urls

,URL
0,https://www.eversports.de/s/seemannsbraut-pole...
1,https://www.eversports.de/s/tsg-blau-gold-ball...
2,https://www.eversports.de/s/pole-stars-osnabr%...
3,https://www.eversports.de/s/aerial-art-studio
4,https://www.eversports.de/s/k%C3%B6rpernah
5,https://www.eversports.de/s/dance-moves-wolfen...
6,https://www.eversports.de/s/mypole
7,https://www.eversports.de/s/tanzstudio-michel
8,https://www.eversports.de/s/pole-sensation
9,https://www.eversports.de/s/poledance-rostock


---

Test V7

In [10]:
result_pole_studio, result_workshops, result_workshop_details, result_urls

(       PoleStudio_Name                                   Adresse    PLZ  \
 0    Tanzstudio Michel         [Schmalweg 50 ,  55252 Wiesbaden]  55252   
 1  Polestation Academy            [Marktstraße 20b ,  6971 Hard]   6971   
 2      Poledance 4 you      [Naugarder Straße 14,  10409 Berlin]  10409   
 3           Luftbonbon  [Königstraße 45 ,  32547 Bad Oeynhausen]  32547   
 
        Stadt               Straße  \
 0  Wiesbaden         Schmalweg 50   
 1       Hard      Marktstraße 20b   
 2     Berlin  Naugarder Straße 14   
 3        Bad       Königstraße 45   
 
                                          Buttons  \
 0             [Übersicht, Klassen, Preise, Team]   
 1     [Übersicht, Klassen, Videos, Preise, Team]   
 2                                    [Übersicht]   
 3  [Übersicht, Klassen, Workshops, Preise, Team]   
 
                             Pole Studio Beschreibung                  E-Mail  \
 0  Das etablierte Tanzstudio in Wiesbaden bietet ...       t.michel.9@gmx.de 

In [11]:
# import pandas as pd
# from a_PyCaller import process_urls
# from tqdm import tqdm
# from datetime import datetime

# def process_and_print_results(url, all_pole_studio_data, all_workshops_data, all_workshop_details_data):
#     data = process_urls([url])

#     if data:
#         for key, df in data.items():
#             if df is not None and not df.empty:
#                 tqdm.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - INFO - Scraping Data from: {url}")
#                 tqdm.write(f"{key.replace('_', ' ').title()}: {len(df)} entries")

#                 # Update the appropriate DataFrame
#                 if key == 'pole_studio_data':
#                     all_pole_studio_data = pd.concat([all_pole_studio_data, df], ignore_index=True)
#                 elif key == 'workshops_data':
#                     all_workshops_data = pd.concat([all_workshops_data, df], ignore_index=True)
#                 elif key == 'workshop_details':
#                     all_workshop_details_data = pd.concat([all_workshop_details_data, df], ignore_index=True)

#     return all_pole_studio_data, all_workshops_data, all_workshop_details_data

# def main():
#     # # Load initial URLs
#     # all_found_urls_s = pd.read_csv("your_output_file.csv")
#     # initial_urls = list(set(all_found_urls_s["0"])) # Delete "[:3]" to scrape all Urls

#     # Initialize DataFrames
#     all_pole_studio_data = pd.DataFrame()
#     all_workshops_data = pd.DataFrame()
#     all_workshop_details_data = pd.DataFrame()

#     # Process each URL with tqdm
#     with tqdm(initial_urls, desc="Processing URLs", dynamic_ncols=True) as pbar:
#         for url in pbar:
#             all_pole_studio_data, all_workshops_data, all_workshop_details_data = process_and_print_results(
#                 url, all_pole_studio_data, all_workshops_data, all_workshop_details_data
#             )

#     # # Export DataFrames to CSV files
#     all_pole_studio_data.to_csv("Pole_Studio_Übersicht_S.csv", index=False)
#     all_workshops_data.to_csv("Workshop_Liste_SW.csv", index=False)
#     all_workshop_details_data.to_csv("Workshop_Übersicht_E.csv", index=False)

#     # Return the final DataFrames
#     return all_pole_studio_data, all_workshops_data, all_workshop_details_data

# # Run the main function and get the final DataFrames
# result_pole_studio, result_workshops, result_workshop_details = main()


In [12]:
# result_pole_studio

---

In [13]:
# result_workshops

In [14]:
# result_workshop_details